<a href="https://colab.research.google.com/github/YolandaMDavis/wildtrack-iqa/blob/task-amenability/task_amenability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym==0.17.3
!pip install stable-baselines==2.10.2
!pip install h5py==3.0.0
!pip install tensorflow==1.13.1
!pip install tensorboard==1.13.1
!pip install Keras==2.2.4
!pip install Keras-Applications==1.0.8
!pip install Keras-Preprocessing==1.1.2
!pip install keras-rl==0.4.2

In [1]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

# get downstream task model
shutil.copy(PARENT_DIR + '/task-amenability/model.pt', 'model.pt')


Mounted at /content/drive


'model.pt'

In [2]:
# clone the repository and move data file
!git clone https://github.com/YolandaMDavis/wildtrack-iqa.git
!mv WildTrack_Raw.zip wildtrack-iqa/.
!mv model.pt wildtrack-iqa/.
%cd "wildtrack-iqa"
!git checkout task-amenability
!mv model.pt taskamenability/taskpredictor/.


Cloning into 'wildtrack-iqa'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 77 (delta 25), reused 58 (delta 13), pack-reused 0
Unpacking objects: 100% (77/77), done.
/content/wildtrack-iqa
Branch 'task-amenability' set up to track remote branch 'task-amenability' from 'origin'.
Switched to a new branch 'task-amenability'


In [3]:
from zipfile import ZipFile
# extract zip file
with ZipFile('WildTrack_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [1]:
%cd "wildtrack-iqa"

import torch
import torchvision
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
import albumentations as A

with open('taskamenability/taskpredictor/class_mapping.json') as data:
    mappings = json.load(data)

class_mapping = {item['model_idx']: item['class_name'] for item in mappings}
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = torch.jit.load('taskamenability/taskpredictor/model.pt').to(device)
transforms = A.load('taskamenability/taskpredictor/transforms.json')

/content/wildtrack-iqa


In [2]:
from torch.utils import data
import csv
import random
import torch
import yaml
import cv2
import os

from time import process_time
from torch.utils.data import Dataset

root_dir = '/content/wildtrack-iqa'
data_dir = root_dir + '/RAW'

def createImageSamples(img_shape, num_samples):
  x = np.random.rand(num_samples, img_shape[0], img_shape[1], img_shape[2])
  y = np.random.randint(low=0, high=2, size=(num_samples, 1))
  return (x,y)

def generate_data_files(sample_percentage=1):

    image_reference_list = []
    subdirectories = list(os.walk(data_dir, topdown=False))[:-1]

    for subdir in subdirectories:
        image_location = subdir[0]
        images = subdir[2]
        species_rating = image_location.rsplit('/', 1)[-1].replace('_', ' ')
        score = int(species_rating.rsplit(' ', 1)[-1])
        species_class = species_rating.rsplit(' ', 1)[:-1][0]
        if len(species_class.rsplit(' ', 1)) > 1:
            species = species_class.rsplit(' ')[0]
            animal_class = ' '.join(species_class.rsplit(' ')[1:])
        else:
            animal_class = 'Unknown'
            species = species_class

        for image in images:
            image_reference = (image_location, species, animal_class, image, score)
            image_reference_list.append(image_reference)
    
    # add percentage of generated data
    samples = int(len(image_reference_list) * .20)
    for i in range(0,samples):
      image_reference_list.append(('sample','sample','sample','sample',0))

    # shuffle then split
    seed = 1234
    random.Random(seed).shuffle(image_reference_list)

    train_index = int(len(image_reference_list) *0.6 *sample_percentage)
    test_index = -int(len(image_reference_list) * 0.2 * sample_percentage) 
    val_index = test_index * 2

    training = image_reference_list[:train_index]
    validation = image_reference_list[val_index:test_index]
    testing = image_reference_list[test_index:]
    return training, validation, testing

def convert_image(img_test):
    image = Image.open(img_test)
    image = image.convert('L')
    image = image.resize((1024, 768))
    image = np.array(image)
    image = np.expand_dims(image, axis=2)
    image = transforms(image=image)['image']
    x = torch.from_numpy(image).to(device)
    return x


def transform_images(images):
  x = []
  y = []
  for img in images:
    if (img[0] == 'sample'):
      x.append(torch.from_numpy(np.random.randint(0, 1, (416, 555, 1))))
      y.append(0)
    else:
      image_name = img[0]+'/'+img[3]
      image = convert_image(image_name)
      x.append(image)
      y.append(1)

  return np.array(x), np.array(y)


In [ ]:
# take a percentage of the full data set as a training/test/validation sample
sample_size=.1
train, valid, test = generate_data_files(sample_size)
x_train, y_train = transform_images(train)
x_val, y_val = transform_images(valid)
x_test, y_test = transform_images(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


**Execute the below installation only in Colab environment**

In [20]:
from taskamenability.interface import PPOInterface

img_shape = (416,555,1)
task_predictor = model
interface = PPOInterface(x_train, y_train, x_val, y_val, x_test, y_test, task_predictor, img_shape)

In [21]:
interface.train(20)

Training started for 100 episodes:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:594.)
  return forward_call(*input, **kwargs)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return forward_call(*input, **kwargs)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: operator() sees varying value in profiling, ignoring and this should be handl

-----------------------------------
| approxkl           | 11.948336  |
| clipfrac           | 0.6422764  |
| explained_variance | -7.68      |
| fps                | 1          |
| n_updates          | 1          |
| policy_entropy     | 1.4189386  |
| policy_loss        | 0.21729243 |
| serial_timesteps   | 123        |
| time_elapsed       | 1.79e-05   |
| total_timesteps    | 123        |
| value_loss         | 8002852.5  |
-----------------------------------


KeyboardInterrupt: ignored

In [9]:
for item in x_val:
  print(item.shape)

torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 555, 3])
torch.Size([416, 5

In [ ]:
# how to extract quality mode from interface and use for prediction
# eventually this should be a pytorch model and we'll need to account